In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.interpolate import NearestNDInterpolator
import glob
import os

# Historical calcs using CRU

In [ ]:
pet_ds = xr.open_dataset('./data/cru_ts4.08.1901.2023.pet.dat.nc')

In [ ]:
pet_ds = pet_ds.sel(time=slice(np.datetime64('2021-01-01'), np.datetime64('2021-12-31'))).mean(dim='time')
# pet_ds = pet_ds.sel(time=slice(np.datetime64('1940-01-01'), np.datetime64('1949-12-31'))).mean(dim='time')

In [ ]:
res_df = pd.read_csv('../remove_bad_water/out/sentinel_2021_v6_wgs84_merged.csv')
res_df = res_df.loc[res_df['hydropoly_max']<100]
res_df['area_ha'] = res_df['area']*100/10000 # HA
res_df['area_km'] = res_df['area']*100/(1000*1000) # km2
res_df['area_m'] = res_df['area']*100
res_df = res_df.loc[res_df['area_ha']<100] # Remove less than 100 ha

In [ ]:
xr_indexer = {
    'lon':xr.DataArray(res_df['longitude'].values, dims=['res']),
    'lat':xr.DataArray(res_df['latitude'].values, dims=['res'])
}

In [ ]:
res_pets = pet_ds.sel(xr_indexer, method='nearest')

In [ ]:
res_pets = res_pets.assign(area_m=('res', res_df['area_m'].values))

In [ ]:
res_month_evap = res_pets['pet']*res_pets['area_m']
res_month_evap = res_month_evap/1000 # Assuming PET is in mm
evap_value = (res_month_evap*365).sum()*1000 # 365 days per year, 1000 converts from cubic meters to liters

In [ ]:
evap_value

# Future Calcs

In [ ]:
def interp(da):
    indices = np.where(np.isfinite(da))
    interp = NearestNDInterpolator(np.transpose(indices), da.data[indices])
    da[...] = interp(*np.indices(da.shape))
    return da


def calc_total_evap(res_pets):
    res_month_evap = res_pets['var']*res_pets['area_m']
    res_month_evap = res_month_evap/1000 # Assuming PET is in mm
    evap_value = (res_month_evap*12).sum()*1000 # 12 months per year
    return evap_value

def full_pet_calcs(res_df, pet_ncdf_path):
    pet_ds = xr.open_dataset(pet_ncdf_path)
    xr_indexer = {
        'longitude':xr.DataArray(res_df['longitude'].values, dims=['res']),
        'latitude':xr.DataArray(res_df['latitude'].values, dims=['res'])
    }
    pet_ds_last10 = pet_ds.sel(time=slice(20901, 209912)).mean(dim='time')
    pet_ds_first10 = pet_ds.sel(time=slice(20101, 202012)).mean(dim='time')
    pet_ds_first10['var'] = interp(pet_ds_first10['var'])
    pet_ds_last10['var'] = interp(pet_ds_last10['var'])
    res_pets_last10 = pet_ds_last10.sel(xr_indexer, method='nearest')
    res_pets_last10 = res_pets_last10.assign(area_m=('res', res_df['area_m'].values))
    res_pets_first10 = pet_ds_first10.sel(xr_indexer, method='nearest')
    res_pets_first10 = res_pets_first10.assign(area_m=('res', res_df['area_m'].values))
    evap_first10 = calc_total_evap(res_pets_first10)
    evap_last10 = calc_total_evap(res_pets_last10)
    return evap_first10, evap_last10

In [ ]:
# Do All and write out results
# First mount using rclone: rclone mount --drive-shared-with-me cu_gdrive: gdrive/
all_models = glob.glob('./data/gdrive/Shared_Kylen_2024/CMIP5_PETpmEstimated/*rcp*.nc')
all_dicts = []
for mod_nc in all_models:
    mod_name = os.path.basename(mod_nc)[:-3]
    print(mod_name)
    baseline, final = full_pet_calcs(res_df, mod_nc)
    out_dict = {
        'model':mod_name,
        'baseline_evap': baseline.values,
        'final_evap': final.values
    }
    all_dicts.append(out_dict)

In [ ]:
# full_df = pd.DataFrame(all_dicts)
# full_df.to_csv('./data/sentinel_v6_2021_allmodels.csv', index=False)
full_df = pd.read_csv('./data/sentinel_v6_2021_allmodels.csv')

In [ ]:
full_df['scenario'] = None
full_df.loc[full_df['model'].str.contains('rcp26'), 'scenario'] = 'rcp26'
full_df.loc[full_df['model'].str.contains('rcp45'), 'scenario'] = 'rcp45'
full_df.loc[full_df['model'].str.contains('rcp85'), 'scenario'] = 'rcp85'
full_df['ratio'] = (full_df['final_evap'] / full_df['baseline_evap'])

In [ ]:
full_df['model']

In [ ]:
full_df.groupby('scenario').aggregate(['mean','std', 'max', 'min', 'count'])['ratio']

In [ ]:
.159737*9.7